In [5]:
import redis
from redis.exceptions import TimeoutError
import cv2
import numpy as np

file_path = '../videos/20_04_2020 TERMINAL - G11.mp4'
img_key = 'sampleA'

client = redis.StrictRedis(host='localhost',
    db=0, socket_connect_timeout=1)

try:
    client.ping()
except TimeoutError:
    print('No hay conexion')

In [6]:
camera = cv2.VideoCapture(file_path)

if not camera.isOpened():
    message = 'No se puede leer el archivo ' + file_path
    print(message)

while True:
    successful_read, raw_frame = camera.read()

    if not successful_read:
        message = 'No se pueden leer mas fotogramas'
        print(message)
        break

    successful_encode, jpeg_frame = cv2.imencode('.jpg', raw_frame)

    if not successful_encode:
        message = 'No se puede codificar el fotograma'
        raise Exception(message)

    client.set(img_key, jpeg_frame.tobytes())

No se pueden leer mas fotogramas


In [1]:
import os
from cams.models import Channel
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Channel.objects.all()

In [35]:
def get_queryset(camera_interface, cameras_number):
    queryset = Channel.objects.filter(camera_interface=camera_interface, state=Channel.STATE_INACTIVE)
    channels_available_count = queryset.count()

    if not channels_available_count:
        print('No hay camaras disponibles para analizar.')
        return queryset

    if channels_available_count < cameras_number:
        print(f'Solo se analizaran {channels_available_count} camaras disponibles.')
        cameras_locked = channels_available_count

    else:
        cameras_locked = cameras_number
    
    new_qs = []
    for channel in queryset[:cameras_locked]:
        channel.state = Channel.STATE_LOCKED
        channel.save(update_fields=['state'])
        new_qs.append(channel)

    return new_qs
        

In [2]:
camera_interface = Channel.REDIS_INTERFACE
server_name = 'virtualdev'
cameras_number = 4
# queryset = Channel.objects.filter(camera_interface=camera_interface, state=Channel.STATE_INACTIVE)
# channels_locked = get_queryset(camera_interface, cameras_number)
channels_locked = Channel.objects.get_and_lock(camera_interface, cameras_number)

Will be analized 1 cameras availables.


In [3]:
process_id = server_name + '.thread01'
[channel.attach(process_id) for channel in channels_locked]

[True]

In [13]:
cameras_locked = len(queryset)

In [14]:
queryset[:cameras_locked]

[<Channel: Terminal>]